In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
906,"Stealing is a crime, and these guys, Kenny Yak...",negative
433,I haven't read the book of this and based on t...,negative
82,"The story-line of ""The Thief of Bagdad"" is com...",positive
552,This movie was by far the worst movie I've eve...,negative
127,I rarely make these comments but I felt compel...,negative


In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...


True

In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
906,stealing crime guy kenny yakkel corbin bernsen...,negative
433,read book based adaptation bother hated every ...,negative
82,story line thief bagdad complex owing told fla...,positive
552,movie far worst movie ever endure believe trie...,negative
127,rarely make comment felt compelled spare other...,negative


In [9]:
df['sentiment'].value_counts()

sentiment
negative    268
positive    232
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
906,stealing crime guy kenny yakkel corbin bernsen...,0
433,read book based adaptation bother hated every ...,0
82,story line thief bagdad complex owing told fla...,1
552,movie far worst movie ever endure believe trie...,0
127,rarely make comment felt compelled spare other...,0


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
from dotenv import load_dotenv
import os

load_dotenv()
mlflow_url = os.getenv("MLFLOWT_TRACKING_URL")
mlflow_repo_name = os.getenv("DAGSHUB_REPO_NAME")
repo_owner= os.getenv("DAGSHUB_REPO_OWNER")

In [18]:
import dagshub

mlflow.set_tracking_uri(mlflow_url)
dagshub.init(repo_owner=repo_owner, repo_name=mlflow_repo_name, mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "nomanmazharr/capstone-mlops"

Repository nomanmazharr/capstone-mlops initialized!

2025/09/09 12:43:12 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3f6d3e788132447295dbf100ea0c55fa', creation_time=1757403793247, experiment_id='0', last_update_time=1757403793247, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [20]:
import mlflow
import logging
import os
import time
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        
        # Save the model locally
        logging.info("Saving and logging the model...")
        model_path = "model.pkl"
        joblib.dump(model, model_path)
        mlflow.log_artifact(model_path)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-09 12:48:30,399 - INFO - Starting MLflow run...


2025-09-09 12:48:30,811 - INFO - Logging preprocessing parameters...
2025-09-09 12:48:31,866 - INFO - Initializing Logistic Regression model...
2025-09-09 12:48:31,867 - INFO - Fitting the model...
2025-09-09 12:48:31,919 - INFO - Model training complete.
2025-09-09 12:48:31,921 - INFO - Logging model parameters...
2025-09-09 12:48:32,332 - INFO - Making predictions...
2025-09-09 12:48:32,334 - INFO - Calculating evaluation metrics...
2025-09-09 12:48:32,353 - INFO - Logging evaluation metrics...
2025-09-09 12:48:36,969 - INFO - Saving and logging the model...
2025-09-09 12:48:39,003 - INFO - Model training and logging completed in 8.19 seconds.
2025-09-09 12:48:39,006 - INFO - Accuracy: 0.68
2025-09-09 12:48:39,008 - INFO - Precision: 0.6612903225806451
2025-09-09 12:48:39,010 - INFO - Recall: 0.6833333333333333
2025-09-09 12:48:39,012 - INFO - F1 Score: 0.6721311475409836


🏃 View run stylish-skunk-925 at: https://dagshub.com/nomanmazharr/capstone-mlops.mlflow/#/experiments/0/runs/f81ce766a0b24e888652b80344b9348e
🧪 View experiment at: https://dagshub.com/nomanmazharr/capstone-mlops.mlflow/#/experiments/0
